In [26]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

# 1. Linear Classifier

In [27]:
# Download the dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print('x_train:', x_train.shape)
print('y_train:',y_train.shape)
print('x_test:',x_test.shape)
print('y_test:',y_test.shape,'\n')

# Classes
K = len(np.unique(y_train)) 

# Data Parameters
Ntr = x_train.shape[0]
Nte = x_test.shape[0]
Din = 3072 # CIFAR10

# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
mean_image = np.mean(x_train, axis=0)
x_train = x_train - mean_image
x_test = x_test - mean_image

# Covert labels to binary classes
y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=K)

# Reshape the data
x_train = np.reshape(x_train,(Ntr,Din))
x_test = np.reshape(x_test,(Nte,Din))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print('x_train reshaped:', x_train.shape)
print('y_train reshaped:',y_train.shape)
print('x_test reshaped:',x_test.shape)
print('y_test reshaped:',y_test.shape)


x_train: (50000, 32, 32, 3)
y_train: (50000, 1)
x_test: (10000, 32, 32, 3)
y_test: (10000, 1) 

x_train reshaped: (50000, 3072)
y_train reshaped: (50000, 10)
x_test reshaped: (10000, 3072)
y_test reshaped: (10000, 10)


In [28]:
# Initialize weights
std=1e-5
w1 = std*np.random.randn(Din, K)
b1 = np.zeros((1,K))
print("w1:", w1.shape)
print("b1:", b1.shape)
batch_size = Ntr

# Initialize parameters for linear model
iterations = 300
lr = 0.014
lr_decay = 0.999
reg = 5e-6 # Regularization parameter - Lambda

# Initialize lists to store history
loss_history = []
train_acc_history = []
val_acc_history = []

w1: (3072, 10)
b1: (1, 10)


### Functions - Linear Model

In [29]:
def LinearLoss(y_p,Y,w1,reg):
    # Calculates loss using the mean sum of squared errors
    dy = y_p - Y # Difference
    batch_size = y_p.shape[0]
    return (1/batch_size)*np.sum(np.square(dy)) + np.sum(w1**2)*reg

In [30]:
def ForwardProp(X,w1,b1):
    # Implements the forward pass of the linear model
    return np.matmul(X,w1)+b1

In [36]:
def BackProp(X,Y,y_p):
    # Calculates dw1 and db1
    dy = 2*(y_p-Y)/X.shape[0] # Derivative
    dw = np.matmul(X.T,dy) ; db = np.sum(dy,axis=0)
    return dw,db

In [32]:
def GradDescent(w1,b1,dw,db,lr,lr_decay):
    # Updates weights and learning rate
    w1 -= dw*lr ;  b1 -= db*lr ; lr *= lr_decay
    return w1,b1,lr

In [33]:
def Accuracy(Y,y_p):
    # Returns a similarity figure of accuracy
    y_p_in = np.argmax(y_p,axis=1)
    y_class = np.argmax(Y,axis=1)
    return np.sum(y_p_in==y_class)*100/y_class.size

In [37]:
seed = 0
rng = np.random.default_rng(seed=seed)

for t in range(iterations):
    indices = np.arange(Ntr)
    rng.shuffle(indices)
    
    # Forward pass
    X = x_train[indices]
    Y = y_train[indices]
    
    y_p = ForwardProp(X,w1,b1)

    # Calculate training loss for each iteration
    loss = LinearLoss(y_p,Y,w1,reg)

    if not((t-1)%10):print('Iteration :',t,'---------- loss',loss)
    
    # Update history
    loss_history.append(loss)
    train_acc_history.append(Accuracy(Y,y_p))

    # Backward pass
    dw,db = BackProp(X,Y,y_p)

    # Perform Gradient Descent
    w1,b1,lr = GradDescent(w1,b1,dw,db,lr,lr_decay)


Iteration : 1 ---------- loss 0.9578989982211862
Iteration : 11 ---------- loss 0.8721104849742985
Iteration : 21 ---------- loss 0.8401459385261969
Iteration : 31 ---------- loss 0.8222232694884302
Iteration : 41 ---------- loss 0.8115404866693534
Iteration : 51 ---------- loss 0.804886157791459
Iteration : 61 ---------- loss 0.8005452032479465
Iteration : 71 ---------- loss 0.797568080538036
Iteration : 81 ---------- loss 0.7954177422692429
Iteration : 91 ---------- loss 0.7937847285349873
Iteration : 101 ---------- loss 0.7924872755938612
Iteration : 111 ---------- loss 0.7914163414431038
Iteration : 121 ---------- loss 0.7905049109487765
Iteration : 131 ---------- loss 0.7897106368710003
Iteration : 141 ---------- loss 0.7890059019124888
Iteration : 151 ---------- loss 0.7883720582186725
Iteration : 161 ---------- loss 0.7877960439864236
Iteration : 171 ---------- loss 0.7872683671530436
Iteration : 181 ---------- loss 0.7867818838846856
Iteration : 191 ---------- loss 0.7863310445

In [38]:
# Printing accuracies and displaying w as images

train_acc_history[-1]

41.942

In [105]:
# Train Accuracy

    print('Train accuracy :',np.sum(y_p_in==y_class)*100/y_class.size)

Train accuracy : 41.952


In [106]:
# Prediction

y_pred = np.matmul(x_test,w1)+b1

print("Learning rate:", lr)
print("Training set loss:", loss_history[-1])

loss_test = (1/Nte)*np.sum(np.square(y_pred - y_test)) + np.sum(w1**2)*reg
print("Test set loss:", loss_test)

y_pred_in = np.argmax(y_pred,axis=1)
y_test_class = np.argmax(y_test,axis=1)

print("Test Accuracy:", np.sum(y_pred_in==y_test_class)*100/y_test_class.size)

Learning rate: 0.010369898450185398
Training set loss: 0.7829008427980855
Test set loss: 0.7876065250735699
Test Accuracy: 40.57


# 2. 2-Layer Neural Network

In [152]:
def sigmoid(z):
  return 1/(1+np.exp(-z))

In [161]:
H = 200
w_1 = std*np.random.randn(Din, H)
b_1 = np.zeros((1,H))
w_2 = std*np.random.randn(H, K)
b_2 = np.zeros((1,K))

print("w1:", w_1.shape)
print("b1:", b_1.shape)
print("w2:", w_2.shape)
print("b2:", b_2.shape)

iterations = 300
lr = 0.014
lr_decay = 0.999
reg = 5e-6
loss_history_nn = []

w1: (3072, 200)
b1: (1, 200)
w2: (200, 10)
b2: (1, 10)


In [155]:
for t in range(iterations):
    indices = np.arange(Ntr)
    rng.shuffle(indices)

    # Forward pass

    Xi = x_train[indices]
    Yi = y_train[indices]
    
    ac = sigmoid(np.matmul(Xi,w_1)+b_1)
    y_nn = np.matmul(ac,w_2)+b_2
    dy = y_nn - Yi
    loss = (1/batch_size)*np.sum(np.square(dy)) + np.sum(w_1**2)*reg
    loss_history_nn.append(loss)
    if not(t%10):print('t :',t,'---------- loss',loss)
    
    # Backward pass
    dac = np.matmul(2*dy,w_2.T)/batch_size
    dw1 = np.matmul(Xi.T,dac*ac*(1-ac))/batch_size + 2*w_1*reg
    db1 = np.sum(dac*ac*(1-ac),axis=0)/batch_size  

    dw2 = np.matmul(ac.T,2*dy)/batch_size + 2*w_2*reg
    db2 = np.sum(2*dy,axis=0)/batch_size  

    # Perform Gradient Descent
    w_1 -= dw1*lr
    b_1 -= db1*lr
    w_2 -= dw2*lr
    b_2 -= db2*lr
    lr *= lr_decay    

t : 0 ---------- loss 0.9999533676809134
t : 10 ---------- loss 0.899999996033859
t : 20 ---------- loss 0.8999999955213841
t : 30 ---------- loss 0.8999999950141696
t : 40 ---------- loss 0.8999999945120051
t : 50 ---------- loss 0.8999999940148401
t : 60 ---------- loss 0.8999999935226263
t : 70 ---------- loss 0.8999999930353132
t : 80 ---------- loss 0.8999999925528522
t : 90 ---------- loss 0.8999999920751947
t : 100 ---------- loss 0.8999999916022934
t : 110 ---------- loss 0.8999999911341006
t : 120 ---------- loss 0.8999999906705691
t : 130 ---------- loss 0.899999990211653
t : 140 ---------- loss 0.8999999897573061
t : 150 ---------- loss 0.8999999893074826
t : 160 ---------- loss 0.8999999888621376
t : 170 ---------- loss 0.8999999884212269
t : 180 ---------- loss 0.8999999879847057
t : 190 ---------- loss 0.8999999875525305
t : 200 ---------- loss 0.8999999871246576
t : 210 ---------- loss 0.8999999867010456
t : 220 ---------- loss 0.8999999862816503
t : 230 ---------- loss 

In [156]:
# Train Accuracy

y_p_in = np.argmax(y_nn,axis=1)
y_class = np.argmax(Yi,axis=1)
print('Train accuracy :',np.sum(y_p_in==y_class)*100/y_class.size)

Train accuracy : 10.054


In [157]:
# Prediction
ac_t = sigmoid(np.matmul(x_test,w_1)+b_1)
y_predic = np.matmul(ac_t,w_2)+b_2

print("Learning rate:", lr)
print("Training set loss:", loss_history[-1])

loss_test = (1/Nte)*np.sum(np.square(y_predic - y_test))
print("Test set loss:", loss_test)

y_predic_in = np.argmax(y_predic,axis=1)
y_test_class = np.argmax(y_test,axis=1)

print("Test Accuracy:", np.sum(y_predic_in==y_test_class)*100/y_test_class.size)

Learning rate: 0.005433502828270419
Training set loss: 0.8999999823415467
Test set loss: 0.8999999823487368
Test Accuracy: 10.05


# 3. Stochastic Gradient Descent

In [172]:
w_1 = std*np.random.randn(Din, H)
b_1 = np.zeros((1,H))
w_2 = std*np.random.randn(H, K)
b_2 = np.zeros((1,K))

print("w1:", w_1.shape)
print("b1:", b_1.shape)
print("w2:", w_2.shape)
print("b2:", b_2.shape)

lr = 0.014
lr_decay = 0.999
reg = 5e-6
loss_history_st = []

batch_size = 500
per_iter = Ntr//batch_size

w1: (3072, 200)
b1: (1, 200)
w2: (200, 10)
b2: (1, 10)


In [175]:
for t in range(iterations):
    indices = np.arange(Ntr)
    rng.shuffle(indices)

    # Forward pass

    Xs = x_train[indices]
    Ys = y_train[indices]

    for iter in range(per_iter):
        batch = Xs[batch_size*iter:batch_size*(iter+1)]
        batchy = Ys[batch_size*iter:batch_size*(iter+1)]    
        ac = sigmoid(np.matmul(batch,w_1)+b_1)
        y_b = np.matmul(ac,w_2)+b_2
        dy = y_b - batchy
        loss = (1/batch_size)*np.sum(np.square(dy)) + np.sum(w_1**2)*reg
        loss_history_st.append(loss)

    
        # Backward pass
        dac = np.matmul(2*dy,w_2.T)/batch_size
        dw1 = np.matmul(batch.T,dac*ac*(1-ac))/batch_size + 2*w_1*reg
        db1 = np.sum(dac*ac*(1-ac),axis=0)/batch_size  

        dw2 = np.matmul(ac.T,2*dy)/batch_size + 2*w_2*reg
        db2 = np.sum(2*dy,axis=0)/batch_size  

        # Perform Gradient Descent
        w_1 -= dw1*lr
        b_1 -= db1*lr
        w_2 -= dw2*lr
        b_2 -= db2*lr
        lr *= lr_decay  

    if not(t%10):print('t :',t,'---------- loss',loss)

t : 0 ---------- loss 0.9008984393065269
t : 10 ---------- loss 0.9010242749428756
t : 20 ---------- loss 0.8998755183687912
t : 30 ---------- loss 0.9001436348757059
t : 40 ---------- loss 0.9002109543043372
t : 50 ---------- loss 0.8999962857309703
t : 60 ---------- loss 0.9000051150120449
t : 70 ---------- loss 0.9000034952564726
t : 80 ---------- loss 0.9000015178841494
t : 90 ---------- loss 0.9000015639762128
t : 100 ---------- loss 0.8999994476612927
t : 110 ---------- loss 0.8999997281218244
t : 120 ---------- loss 0.8999998851480435
t : 130 ---------- loss 0.8999995449923835
t : 140 ---------- loss 0.9000002313405648
t : 150 ---------- loss 0.899999506711902
t : 160 ---------- loss 0.8999999388771533
t : 170 ---------- loss 0.9000000761329955
t : 180 ---------- loss 0.9000003026589055
t : 190 ---------- loss 0.8999995463692197
t : 200 ---------- loss 0.9000002005616976
t : 210 ---------- loss 0.9000007463238083
t : 220 ---------- loss 0.8999996852648459
t : 230 ---------- loss

KeyboardInterrupt: 

In [174]:
dy.shape

(50000, 10)